<h1>CS689A: Computational Linguistics for Indian Languages</h1>
<h2>Assignment-2</h2>
<h3>Question-2</h3>

**Dependencies:** `numpy, conllu, datasets, transformers, seqeval`

### Creating train-validation-test splits
> Replace the file accordingly

**Note: Skip this section if files are already available in form of train-val-test splits**

from conllu import parse

data_file = open("validation.conllu", "r", encoding="utf-8").read()
sentences = parse(data_file)
len(sentences)

sentences = [sentence.serialize() for sentence in sentences]

from sklearn.model_selection import train_test_split

## Split size may be adjusted here
train, test = train_test_split(sentences,test_size=0.20)
train, val = train_test_split(train,test_size=0.10)

len(train),len(val),len(test)

> Storing the outputs in `data/` folder

import os

## Create directory data/ if does not exist
if not os.path.exists('data'):
    os.makedirs('data')
    
for l,s in zip([train,val,test],['train','validation','test']):
    with open(f'data/{s}.conllu','w',encoding='utf-8') as fp:
        fp.write(''.join(l))

### Tokenization

The following section does tokenization for UPOS tagging task

> Loading data

In [1]:
from datasets import load_dataset

2022-11-07 22:03:27.694558: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-07 22:03:28.552045: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-07 22:03:28.552078: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-07 22:03:28.651924: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-07 22:03:31.209660: W tensorflow/stream_executor/platform/de

**Note: If the above convention for saving files was not followed, the file `ConlluIndic.py` needs to be editied so that `__TRAINING_FILE__` etc. point correctly**

In [2]:
dataset = load_dataset('ConlluIndic.py')
dataset

Found cached dataset conllu_indic (/home/sunild/.cache/huggingface/datasets/conllu_indic/conlluindic/1.0.0/96f4ae3d955b437d53ab745469050ad8372e37a72fcca78066bab97f7824c734)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'lemmas', 'upos_tags'],
        num_rows: 1194
    })
    validation: Dataset({
        features: ['id', 'tokens', 'lemmas', 'upos_tags'],
        num_rows: 133
    })
    test: Dataset({
        features: ['id', 'tokens', 'lemmas', 'upos_tags'],
        num_rows: 332
    })
})

> Tokenization

In [3]:
from transformers import AutoTokenizer

model_checkpoint = 'ai4bharat/indic-bert'

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/home/sunild/anaconda3/lib/python3.8/site-packages/packaging/version.py:111: DeprecationWarning: Creating a LegacyVersion has been deprecated and will be removed in the next major release
  warnings.warn(
/home/sunild/.local/lib/python3.8/site-packages/transformers/convert_slow_tokenizer.py:308: ResourceWarning: unclosed file <_io.BufferedReader name='/home/sunild/.cache/huggingface/transformers/97e976f62c951292c6fdb2bb5bf1f34f5450585dda98be8dedea2b97e4f25b86.5c8e83e7afe2b2faae1dc1a6b0fcdb911480df6af7079e9df53aa83ed0763de2'>
  m.ParseFromString(open(self.original_tokenizer.vocab_file, "rb").read())


In [4]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            #if label % 2 == 1:
            #    label += 1
            new_labels.append(label)

    return new_labels


In [5]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["upos_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [6]:
tokenized_datasets = dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=dataset["train"].column_names,
)

Loading cached processed dataset at /home/sunild/.cache/huggingface/datasets/conllu_indic/conlluindic/1.0.0/96f4ae3d955b437d53ab745469050ad8372e37a72fcca78066bab97f7824c734/cache-48c5b360a7cec714.arrow
Loading cached processed dataset at /home/sunild/.cache/huggingface/datasets/conllu_indic/conlluindic/1.0.0/96f4ae3d955b437d53ab745469050ad8372e37a72fcca78066bab97f7824c734/cache-c254295978b886ad.arrow
Loading cached processed dataset at /home/sunild/.cache/huggingface/datasets/conllu_indic/conlluindic/1.0.0/96f4ae3d955b437d53ab745469050ad8372e37a72fcca78066bab97f7824c734/cache-f228ac7bb92c64af.arrow


In [7]:
tokenized_datasets.save_to_disk('data/tokenized_datasets/')

/home/sunild/anaconda3/lib/python3.8/site-packages/datasets/dataset_dict.py:1075: ResourceWarning: unclosed file <_io.TextIOWrapper name='/home/sunild/Downloads/CS689A/asgn2/data/tokenized_datasets/dataset_dict.json' encoding='utf-8'>
  json.dump(


### Fine-tuning and Evaluation

In [8]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, DataCollatorForTokenClassification, TrainingArguments, Trainer
from datasets import load_metric, load_from_disk, load_dataset
import numpy as np

> Loads the model, tokenizer, data collator, metric and tokenized datasets

**Note: Don't forget to edit the file paths if the previous conventions were not followed**

In [9]:
model_checkpoint = 'ai4bharat/indic-bert'


tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenized_datasets = load_from_disk('data/tokenized_datasets/')



data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
metric = load_metric('seqeval')


dataset = load_dataset('ConlluIndic.py')

upos_feature = dataset['train'].features['upos_tags']
label_names = upos_feature.feature.names



id2label = {str(i): label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}


model = AutoModelForTokenClassification.from_pretrained(model_checkpoint,id2label=id2label,label2id=label2id)

/home/sunild/.local/lib/python3.8/site-packages/transformers/convert_slow_tokenizer.py:308: ResourceWarning: unclosed file <_io.BufferedReader name='/home/sunild/.cache/huggingface/transformers/97e976f62c951292c6fdb2bb5bf1f34f5450585dda98be8dedea2b97e4f25b86.5c8e83e7afe2b2faae1dc1a6b0fcdb911480df6af7079e9df53aa83ed0763de2'>
  m.ParseFromString(open(self.original_tokenizer.vocab_file, "rb").read())
/home/sunild/anaconda3/lib/python3.8/site-packages/datasets/dataset_dict.py:1120: ResourceWarning: unclosed file <_io.TextIOWrapper name='/home/sunild/Downloads/CS689A/asgn2/data/tokenized_datasets/dataset_dict.json' encoding='utf-8'>
  for k in json.load(fs.open(dataset_dict_json_path, "r", encoding="utf-8"))["splits"]:
/tmp/ipykernel_4352/3837697733.py:10: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('seqeval')
Found

  0%|          | 0/3 [00:00<?, ?it/s]

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertForTokenClassification: ['predictions.bias', 'predictions.LayerNorm.weight', 'predictions.LayerNorm.bias', 'predictions.dense.weight', 'predictions.dense.bias', 'predictions.decoder.weight', 'predictions.decoder.bias', 'sop_classifier.classifier.weight', 'sop_classifier.classifier.bias']
- This IS expected if you are initializing AlbertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForTokenClassification were not initialized from the model checkpoint at ai4bharat/indic-bert and a

> Metric computation

In [10]:
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    # save predictions and labels for debugging
    with open('predictions.txt', 'w') as f:
        f.write(str(true_predictions))

    with open('labels.txt', 'w') as f:
        f.write(str(true_labels))

    # save metrics to file
    with open('metrics.txt', 'w') as f:
        f.write(str(all_metrics))
        
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }


> Defining the trainer

`fp16 = True` may be set for GPU

Similarly `batch_size` can be increased

In [11]:
batch_size = 15
args = TrainingArguments(
    output_dir=f"model/upos",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_strategy="epoch",
    learning_rate=2e-3,
    num_train_epochs=1,
    weight_decay=0.01,
    save_total_limit=1,
    #fp16= True,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    #eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)


> Train the model

In [17]:
trainer.

TypeError: log_metrics() missing 2 required positional arguments: 'split' and 'metrics'

In [12]:
trainer.train()

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

TypeError: classification_report() got an unexpected keyword argument 'output_dict'

> Evaluate the model

**Note: Make sure evaluation results are reported on test dataset, for this training args should be changed**

In [18]:
trainer.evaluate()

TypeError: classification_report() got an unexpected keyword argument 'output_dict'